# Final Project - Part 4 - Model Draft 5 (Demo + RR vs Type of Sex Exp)

# 5) Mutinomial Logistic Regression (Input: Demographics & Recruiter Relation; Output: Type of Sexual Exploitation)

In [164]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.cross_validation import cross_val_score,train_test_split
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
from sklearn.base import BaseEstimator
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.multiclass import OneVsRestClassifier
import random
import matplotlib as plt
import pdb

In [165]:
#Read Cleaned Global Data
GD_data_final = pd.read_csv('assets/GD_data_final.csv')
GD_data_final.head()

,yearOfRegistration,gender,majorityStatus,ageBroad_mid,citizenshipRegion,citizenshipGeoCategory,exploitationRegion,exploitationGeoCategory,mocDebtBondage,mocTakesEarnings,...,tosProstitution,tosPornography,tosRemoteInteractiveServices,tosPrivateSexualServices,tosNotSpecified,rrIntimatePartner,rrFriend,rrFamily,rrOther,rrUnknown
0,2010,Female,Adult,19.0,Central_Asia,Asia,Central_Asia,Asia,0,0,...,0,0,0,0,1,0,0,0,0,1
1,2004,Female,Adult,19.0,Eastern_Europe,Europe,Eastern_Europe,Europe,0,0,...,0,0,0,0,1,0,0,0,0,1
2,2010,Female,Adult,19.0,Central_Asia,Asia,Central_Asia,Asia,1,0,...,0,0,0,0,1,0,0,0,0,1
3,2010,Female,Adult,19.0,Central_Asia,Asia,Central_Asia,Asia,1,0,...,0,0,0,0,1,0,0,0,0,1
4,2012,Female,Minor,13.0,North_America,North_America,North_America,North_America,0,0,...,0,0,0,0,0,0,0,1,0,0


In [166]:
GD_data_final.columns

Index([u'yearOfRegistration', u'gender', u'majorityStatus', u'ageBroad_mid',
       u'citizenshipRegion', u'citizenshipGeoCategory', u'exploitationRegion',
       u'exploitationGeoCategory', u'mocDebtBondage', u'mocTakesEarnings',
       u'mocRestrictsFinancialAccess', u'mocThreats', u'mocPsychologicalAbuse',
       u'mocPhysicalAbuse', u'mocSexualAbuse', u'mocFalsePromises',
       u'mocPsychoactiveSubstances', u'mocRestrictsMovement',
       u'mocRestrictsMedicalCare', u'mocExcessiveWorkingHours',
       u'mocUsesChildren', u'mocThreatOfLawEnforcement',
       u'mocWithholdsNecessities', u'mocWithholdsDocuments', u'mocOther',
       u'mocNotSpecified', u'isForcedLabour', u'isSexualExploit',
       u'isOtherExploit', u'isSexAndLabour', u'isForcedMarriage',
       u'isForcedMilitary', u'isOrganRemoval', u'tolAgriculture',
       u'tolAquafarming', u'tolBegging', u'tolConstruction',
       u'tolDomesticWork', u'tolHospitality', u'tolIllicitActivities',
       u'tolManufacturing', u'tolM

In [219]:
# Read Demographic vs Labor Type Data - Only for isForcedLabour == 1
demo_sexExptype_calc = pd.read_csv('assets/demo_sexExptype_calc.csv')
demo_sexExptype_calc.reset_index(drop=True)
demo_sexExptype_calc.head()

,yearOfRegistration,ageBroad_mid,g_Female,g_Male,ms_Adult,cr_Central_Asia,cr_East_Africa,cr_East_Asia,cr_Eastern_Europe,cr_Middle_East,...,cg_Europe,cg_Middle_East,cg_North_America,cg_South_America,cg_Unknown,tosProstitution,tosPornography,tosRemoteInteractiveServices,tosPrivateSexualServices,tosNotSpecified
0,2010,19.0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2004,19.0,1,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1
2,2010,19.0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2010,19.0,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,2002,34.0,1,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,1


In [220]:
demo_sexExptype_calc.columns
len(demo_sexExptype_calc)

11134

In [221]:
# Carve out Just Recruiter Relation data
GD_data_Sex = GD_data_final[GD_data_final['isSexualExploit']==1].reset_index(drop=True)
rr_sexType_handCalc = GD_data_Sex[['rrIntimatePartner','rrFriend','rrFamily','rrOther','rrUnknown']]
rr_sexType_handCalc.reset_index(drop=True)
rr_sexType_handCalc.head()
len(rr_sexType_handCalc)

11134

In [223]:
## JOIN Demographics, Means of Control, and Recruiter Relation data
#demo_rr_labortype_calc = demo_labortype_calc.join(rr_Labor_handCalc)
demo_rr_sextype_calc = pd.concat([demo_sexExptype_calc,rr_sexType_handCalc],axis=1)
demo_rr_sextype_calc.head()
demo_rr_sextype_calc.columns

Index([u'yearOfRegistration', u'ageBroad_mid', u'g_Female', u'g_Male',
       u'ms_Adult', u'cr_Central_Asia', u'cr_East_Africa', u'cr_East_Asia',
       u'cr_Eastern_Europe', u'cr_Middle_East', u'cr_North_America',
       u'cr_Northeast_Africa', u'cr_South_America', u'cr_South_Asia',
       u'cr_Southeast_Asia', u'cr_Southeastern_Europe', u'cr_Unknown',
       u'cr_West_Africa', u'cg_Africa', u'cg_Asia', u'cg_Europe',
       u'cg_Middle_East', u'cg_North_America', u'cg_South_America',
       u'cg_Unknown', u'tosProstitution', u'tosPornography',
       u'tosRemoteInteractiveServices', u'tosPrivateSexualServices',
       u'tosNotSpecified', u'rrIntimatePartner', u'rrFriend', u'rrFamily',
       u'rrOther', u'rrUnknown'],
      dtype='object')

In [224]:
### Consolidate all types of labor into one column

def combine_sexExpType(x):
    if x['tosProstitution'] == 1:
        return 'Prostitution'
    elif x['tosPornography'] == 1:
        return 'Pornography'
    elif x['tosRemoteInteractiveServices'] == 1:
        return 'RemoteInteractiveServices'
    elif x['tosPrivateSexualServices'] == 1:
        return 'PrivateSexualServices'
    elif x['tosNotSpecified'] == 1:
        return 'NotSpecified'
    else:
        return 0
        
demo_rr_sextype_calc['SexExpType'] = demo_rr_sextype_calc.apply(combine_sexExpType,axis=1)
demo_rr_sextype_calc.head()


,yearOfRegistration,ageBroad_mid,g_Female,g_Male,ms_Adult,cr_Central_Asia,cr_East_Africa,cr_East_Asia,cr_Eastern_Europe,cr_Middle_East,...,tosPornography,tosRemoteInteractiveServices,tosPrivateSexualServices,tosNotSpecified,rrIntimatePartner,rrFriend,rrFamily,rrOther,rrUnknown,SexExpType
0,2010,19.0,1,0,1,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,NotSpecified
1,2004,19.0,1,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,NotSpecified
2,2010,19.0,1,0,1,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,NotSpecified
3,2010,19.0,1,0,1,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,NotSpecified
4,2002,34.0,1,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,NotSpecified


In [225]:
demo_rr_sextype_calc[['SexExpType','tosNotSpecified']].groupby('SexExpType').count()

,tosNotSpecified
SexExpType,
NotSpecified,6776
Pornography,46
PrivateSexualServices,9
Prostitution,4303


In [236]:
# Try OneVsRestClassifier - Multilabel classification method
def main():
    #headers = ['ageBroad_mid', 'g_Female', 'ms_Adult', 
                              #'cr_Central_Asia', 'cr_East_Africa', 'cr_East_Asia', 'cr_Eastern_Europe', 'cr_Middle_East', 
                              #'cr_North_America', 'cr_Northeast_Africa', 'cr_South_America', 'cr_South_Asia', 
                              #'cr_Southeast_Asia', 'cr_Southeastern_Europe', 'cr_West_Africa', 
                              #'cg_Africa', 'cg_Asia', 'cg_Europe', 'cg_Middle_East', 'cg_North_America', 'cg_South_America', 
                                #'rrIntimatePartner', 'rrFriend', 'rrFamily', 'rrOther',
                              # 'SexExpType']
    headers = [#'ms_Adult',#'ageBroad_mid','g_Female',
                              'cr_Central_Asia', 'cr_East_Africa', 'cr_East_Asia', 'cr_Eastern_Europe', 'cr_Middle_East', 
                              'cr_North_America', 'cr_Northeast_Africa', 'cr_South_America', 'cr_South_Asia', 
                              'cr_Southeast_Asia', 'cr_Southeastern_Europe', 'cr_West_Africa', 
                              #'cg_Africa', 'cg_Asia', 'cg_Europe', 'cg_Middle_East', 'cg_North_America', 'cg_South_America', 
                                #'rrIntimatePartner', 'rrFriend', 'rrFamily', 'rrOther', 
                                 'SexExpType']
    
    ### Train Test Split - Out put col index -1 
    X = demo_rr_sextype_calc[headers[:-1]]
    y = demo_rr_sextype_calc[headers[-1]]
    train_x, test_x, train_y, test_y = train_test_split(X, y, train_size=0.7)
    
   # USE TWO DIFFERENT APPROACHES TO TRAIN 
    # Train multi-classification model with logistic regression
    lr = LogisticRegression(C=10)
    lr.fit(train_x, train_y)
 
    # Train multinomial logistic regression model
    mul_lr = LogisticRegression(C=10,multi_class='multinomial', solver='newton-cg')
    mul_lr.fit(train_x, train_y)
 
    print "Logistic regression Train Accuracy :: ", metrics.accuracy_score(train_y, lr.predict(train_x))
    print "Logistic regression Test Accuracy :: ", metrics.accuracy_score(test_y, lr.predict(test_x))
 
    print "Multinomial Logistic regression Train Accuracy :: ", metrics.accuracy_score(train_y, mul_lr.predict(train_x))
    print "Multinomial Logistic regression Test Accuracy :: ", metrics.accuracy_score(test_y, mul_lr.predict(test_x))
    
    print ("Cross validating...")

    print ("Multi-classification Log Reg - train accuracy: ")
    print (np.mean(cross_val_score(lr, train_x, train_y.values, scoring='accuracy', cv=10)))
    print ("Multinomial Log Reg - train accuracy: ")
    print (np.mean(cross_val_score(mul_lr, train_x, train_y.values, scoring='accuracy', cv=10)))
    
    region_unit = np.identity(12)
    #TEST  AGE-GENDER COMBO:
    #ag = np.array([[10,1],[10,0],[22,1],[22,0],[40,1],[40,0]])
    #age = np.array([[10],[25],[35],[45]])
    
    
    print (mul_lr.get_params())
    print ("X", X.columns)
    #print ("Test Age: ", age)
    #print ("Test Female, Male")
    print (mul_lr.predict(region_unit))
    
if __name__ == '__main__':
  main()

Logistic regression Train Accuracy ::  0.687283459515
Logistic regression Test Accuracy ::  0.68093385214
Multinomial Logistic regression Train Accuracy ::  0.687283459515
Multinomial Logistic regression Test Accuracy ::  0.68093385214
Cross validating...
Multi-classification Log Reg - train accuracy: 
0.687275713999
Multinomial Log Reg - train accuracy: 
0.687275713999
{'warm_start': False, 'C': 10, 'n_jobs': 1, 'verbose': 0, 'intercept_scaling': 1, 'fit_intercept': True, 'max_iter': 100, 'penalty': 'l2', 'multi_class': 'multinomial', 'random_state': None, 'dual': False, 'tol': 0.0001, 'solver': 'newton-cg', 'class_weight': None}
('X', Index([u'cr_Central_Asia', u'cr_East_Africa', u'cr_East_Asia',
       u'cr_Eastern_Europe', u'cr_Middle_East', u'cr_North_America',
       u'cr_Northeast_Africa', u'cr_South_America', u'cr_South_Asia',
       u'cr_Southeast_Asia', u'cr_Southeastern_Europe', u'cr_West_Africa'],
      dtype='object'))
['NotSpecified' 'Prostitution' 'Prostitution' 'NotSpec

## Observations: Accuracy Scores - Sex Exploitation Type

### All Demo & RR features with Citizenship Region:
Multinomial Logistic regression Train Accuracy ::  0.691518029
Multinomial Logistic regression Test Accuracy ::  0.687219395391

### ageBroad_mid :
Multinomial Logistic regression Train Accuracy ::  0.607339920441
Multinomial Logistic regression Test Accuracy ::  0.611493564801

### g_Female:
Multinomial Logistic regression Train Accuracy ::  0.610676247915
Multinomial Logistic regression Test Accuracy ::  0.603711463634

### ms_Adult:
Multinomial Logistic regression Train Accuracy ::  0.646092647248
Multinomial Logistic regression Test Accuracy ::  0.651302005388

### citizenship Regions :
Multinomial Logistic regression Train Accuracy ::  0.687283459515
Multinomial Logistic regression Test Accuracy ::  0.68093385214

### recruiter relation :
Multinomial Logistic regression Train Accuracy ::  0.675132275132
Multinomial Logistic regression Test Accuracy ::  0.705185185185

Conclusions?  
Age - not indicative
Minor - Prostitution
East Africa - Prostitution
East Asia - Prostitution
Middle East - Prostitution
Northeast Africa - Prostitution
South Asia and Southeast Asia - Prostitution

